# Lab: Stock Prediction<br><font size="4">**Portfolio Management**</font>
***Series: Deep learning with Tensorflow 2 and Keras***<br>
*Prepared by: Jacob Cybulski*<br>
*Date: Sept 2020*

***Overview:***<br>
*This notebook demonstrates how to use Recurrent Neural Network (such as LSTM and GRU) to predict a stock value based on the movement of several stocks in a portfolio, e.g. HPQ, GOOGL, MSFT, IBM, INTC, ADBE, AMZN and AAPL, the last one we will be trying to predict.*

<hr style="height:1px;border:none;color:#333;background-color:#333;" />

**Enter your name and student number below, e.g.**

<font color='red'>**Your name and student number go here, e.g. Jacob Cybulski (12345678)**</font>

# Lab exercises / Assignment 3 Part C

**This lab exercise and assignment follow the example demonstrated in lecture.<br>
It's aim is to create a new model and alter it to improve performance.**

*Undertake the following tasks.*

1. Run this notebook and record results (multivariate to predict IBM), report
2. Run this notebook in a univariate mode (use IBM but also try another stock), compare and report
3. Optimise multivariate network using various optimisers and parameters, compare and report
4. Use multiple stocks as part of the label, experiment with different network architectures, report
5. Then optimise the selected architecture for the multi-label case, compare and report
   - Does it matter if the custom loss function was used or not?
6. Check if your best network overall is capable of predicting 7 days ahead (instead of 2), report
7. Provide analysis and recommendations for the best solution in "portfolio management"
8. Reflect on your experience gained in module 3 on Deep Learning
9. Challenges for Python hackers (just for glory not marks)
   - Calculate errors in true MAE units (unscaled)
   - Exclude interpolated data from error calculation (these are not true data)

*Keep the record of your activities, models and results in the section at the*
<font color="red">**end of this notebook**</font>.
<hr style="height:1px;border:none;color:#333;background-color:#333;" />

### Preparation

*General purpose libraries*

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os
import math

*Sci-kit Learn libs*

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error

*Deep learning libraries*

In [ ]:
import tensorflow as tf
from tensorflow.keras import metrics
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, GRU, Embedding, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Activation
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop, SGD, Adadelta
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.backend import square, mean
from tensorflow.keras.models import load_model

*Options to control display of information*

In [ ]:
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
plt.rcParams["figure.figsize"] = [10, 5]

*Check GPU on this machine*

In [ ]:
tf.config.list_physical_devices('GPU')

### Utilities

*Allows display of data frames side by side*

In [ ]:
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

*Collect history*

In [ ]:
# Initiates collections of model performance
def start_hist():
    return {}

# Adds more performance indicators to history
def collect_hist(accum_hist, next_hist):
    # Get all keys
    keys = list(next_hist.keys())
    for k in keys:
        if k in accum_hist:
            accum_hist[k].extend(next_hist[k])
        else:
            accum_hist[k] = next_hist[k]
    return accum_hist

*Plotting history*

In [ ]:
def plot_hist(h, xsize=6, ysize=5):
    # Prepare plotting
    fig_size = plt.rcParams["figure.figsize"]
    plt.rcParams["figure.figsize"] = [xsize, ysize]
    
    # Get training and validation keys
    ks = list(h.keys())
    n2 = math.floor(len(ks)/2)
    train_keys = ks[0:n2]
    valid_keys = ks[n2:2*n2]
    
    # summarize history for different metrics
    for i in range(n2):
        plt.plot(h[train_keys[i]])
        plt.plot(h[valid_keys[i]])
        plt.title('Training vs Validation '+train_keys[i])
        plt.ylabel(train_keys[i])
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.draw()
        plt.show()
    
    return

### Load data

*All available stock data*

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
stock = pd.read_csv('../input/stocks.csv')
stock['Date'] = pd.to_datetime(stock['Date'])
print('Stock data shape: ', stock.shape)
print('Date from: ', stock['Date'].min(), 'to: ', stock['Date'].max())
print('Day from: ', stock['Day'].min(), 'to: ', stock['Day'].max())
print('Number of records:', stock.shape[0], 'max-min day:', stock['Day'].max()-stock['Day'].min()+1)
stock.head()

*Plot a couple of stocks*

In [ ]:
ax1 = stock['AAPL'].plot(color="lightblue", alpha=0.9)
ax1.set(xlabel='Days', ylabel='Stock Value')
stock['IBM'].plot(color="red", alpha=0.5, ax=ax1)
plt.legend(['AAPL', 'BLUE'], loc='upper left')

## Data preparation

### Data selection

*Select date, day and stocks for the portfolio, remember that Date and Day are needed here.*

In [ ]:
stocks = ['Date', 'Day', 'AAPL', 'ADBE', 'AMZN', 'GOOGL', 'IBM', 'MSFT'] # Try just IBM
labels = ['IBM', 'AAPL'] # Try others, e.g. AAPL or HPQ (but add it above)
horizon = 2

# Select raw_portfolio
raw_portfolio = stock[stocks]

print('Selected stocks: ', ', '.join(stocks))
print('Prediction target: ', labels)
print('Selected data shape: ', raw_portfolio.shape)
raw_portfolio.describe(include='all')

### Imputation of missing values (if needed)

*Note that we need to check if the stock data has any missing data.*<br>

In [ ]:
print('Records:\t', len(raw_portfolio))
missing = raw_portfolio.isnull().sum()
print(missing.sort_values(ascending=False))

### Resample and interpolate missing values

*We need to resample the contents of all Pandas (sub) data-frames by removing empty rows and columns (if any),<br>
and then up-sampling the data frame by interpolating records in 1-day intervals.*

In [ ]:
def stock_resample(df):

    # Remove all empty rows.
    df_res = df.dropna(how='all')
    
    # Create a sorted index from date
    df_res['Date'] = pd.to_datetime(df_res['Date'])
    df_res.index = df_res['Date']
    df_res.sort_index(inplace=True)
    del df_res['Date']

    # Upsample so the time-series has data for every day
    df_res = df_res.resample('D')

    # Fill in missing values by interpolating in between existing records
    df_res = df_res.interpolate(method='values')

    # Remove any empty rows that may have been created in this process
    df_res = df_res.dropna(how='all')
    
    # Columns no longer needed
    del df_res['Day']

    return df_res

*Resample all stocks*

In [ ]:
portfolio_x = stock_resample(raw_portfolio)

*Observe missing dates in the original table - we should have no missing values*

In [ ]:
raw_portfolio[(raw_portfolio['Date'] >= '2010-01-07') & (raw_portfolio['Date'] <= '2010-01-12')]

*Observe that missing values have been interpolated*

In [ ]:
portfolio_x['2010-01-07':'2010-01-12']

*The labels come from the original data but time-shifted to the left (past)*

In [ ]:
portfolio_y = portfolio_x[labels].shift(-horizon)

*As we shift data to the left (past), examples from the beginning of the series will drop off, and those from the end will become undefined*

In [ ]:
display_side_by_side(
    pd.concat([portfolio_x[labels].head(horizon + 5)], keys=['Original series'], axis=1),
    pd.concat([portfolio_y.head(5)], keys=['Start of new series'], axis=1),
    pd.concat([portfolio_y.tail(horizon+5)], keys=['End of new series'], axis=1))

### Data preparation for Keras RNN (NumPy arrays)

*We now convert the Pandas data-frames to NumPy arrays that can be input to the neural network. We also remove the last part of the numpy arrays, because the target-data has `NaN` for the shifted period, and we only want to have valid data and we need the same array-shapes for the input- and output-data.*

In [ ]:
# Correct predictors x and targets/labels y for the horizon shift
x_data = portfolio_x.values[0:-horizon]
y_data = portfolio_y.values[:-horizon]

# Calculate training and testing partition sizes
num_data = len(x_data)
train_split = 0.7
num_train = int(train_split * num_data)
num_test = num_data - num_train

# Define boundaries for training and testing
x_train = x_data[0:num_train]
x_test = x_data[num_train:]
y_train = y_data[0:num_train]
y_test = y_data[num_train:]

# Identify time events to be used in training
num_x_events = x_data.shape[1]
num_y_events = y_data.shape[1]

print("Original x shape:", x_data.shape, ", New x shape:", x_train.shape, x_test.shape)
print("Original y shape:", y_data.shape, ", New y shape:", y_train.shape, y_test.shape)

### Scale data

*Before this data can be used by neural net, its X and Y values need scaling to [0:1] range.*
*We will need to estimate the future stock growth across the portfolio and scale data to a smaller range, e.g. [0:0.4], then we will apply the scaler to all new data without any clipping (which could cause problems).*<br>
<font color="red">*We must check that the new/test data does not go (much) over the [0:1] range!*</font>

In [ ]:
print("Before train x scaling - Min:", np.min(x_train), ", Max:", np.max(x_train))
print("Before test x scaling - Min:", np.min(x_test), ", Max:", np.max(x_test))
x_scaler = MinMaxScaler(feature_range=(0, 0.3), copy=True)
x_train_scaled = x_scaler.fit_transform(x_train).clip(0, 1)
x_test_scaled = x_scaler.transform(x_test).clip(0, 1)
print("After train x scaling - Min:", np.min(x_train_scaled), ", Max:", np.max(x_train_scaled))
print("After test x scaling - Min:", np.min(x_test_scaled), ", Max:", np.max(x_test_scaled))

In [ ]:
print("Before train y scaling - Min:", np.min(y_train), ", Max:", np.max(y_train))
print("Before test y scaling - Min:", np.min(y_test), ", Max:", np.max(y_test))
y_scaler = x_scaler
y_train_scaled = y_scaler.fit_transform(y_train)
y_test_scaled = y_scaler.transform(y_test)
print("After train y scaling - Min:", np.min(y_train_scaled), ", Max:", np.max(y_train_scaled))
print("After test y scaling - Min:", np.min(y_test_scaled), ", Max:", np.max(y_test_scaled))

In [ ]:
# Print a week of Xs and Ys side by side
display_side_by_side(
    pd.concat([pd.DataFrame(x_train_scaled, columns=portfolio_x.columns).head(7)], keys=['X'], axis=1),
    pd.concat([pd.DataFrame(y_train_scaled, columns=portfolio_y.columns).head(7)], keys=['Y'], axis=1))

## Create a data generator

*As the training-data could potentially have 1000s of observations, so instead of training the neural net on the complete sequences, we will use a generator function to create batches of shorter sub-sequences picked at random from training data.*

*Use a large batch-size to keep the GPU busy. Or we can pick a smaller batch size to improve accuracy and performance. You may have to adjust this number depending on your GPU, its RAM and your sequence_length.

In [ ]:
batch_size = 256
sequence_length = 42 # 6 weeks x 7 days

*Generator function*

In [ ]:
def batch_generator(batch_size, sequence_length, nt, nx, ny):
    while True:
        # Allocate a new array for the batch of input-events.
        x_shape = (batch_size, sequence_length, nx)
        x_batch = np.zeros(shape=x_shape, dtype=np.float16)

        # Allocate a new array for the batch of output-events.
        y_shape = (batch_size, sequence_length, ny)
        y_batch = np.zeros(shape=y_shape, dtype=np.float16)

        # Fill the batch with random sequences of data.
        for i in range(batch_size):
            # Get a random start-index.
            # This points somewhere into the training-data.
            idx = np.random.randint(nt - sequence_length)
            
            # Copy the sequences of data starting at this index.
            x_batch[i] = x_train_scaled[idx:idx+sequence_length]
            y_batch[i] = y_train_scaled[idx:idx+sequence_length]
        
        yield (x_batch, y_batch)

*Create the batch-generator*

In [ ]:
# fix random seed for (near) reproducibility
np.random.seed(7)

generator = batch_generator(batch_size=batch_size, sequence_length=sequence_length, 
                            nt=num_train, nx=num_x_events, ny=num_y_events)

*Create validation set*

Note: Model validation will be performed after each epoch and when the model improves its weights will be saved.<br>
For training, we will use the batch-generator, however, for validation we will use the entire sequence, i.e. including x and y.

In [ ]:
validation_data = (np.expand_dims(x_test_scaled, axis=0),
                   np.expand_dims(y_test_scaled, axis=0))

*Test generator*

In [ ]:
batch = 0  # Some sequence in the batch
event = 0  # Some event from input-events

x_batch, y_batch = next(generator)
print('X batch shape =', x_batch.shape, ', Input size =', x_batch.shape[1]*x_batch.shape[2])
print('Y batch shape =', y_batch.shape, ', Output size =', y_batch.shape[1]*y_batch.shape[2])

x_seq = x_batch[batch, :, event]
y_seq = y_batch[batch, :, event]
plt.plot(x_seq)
plt.plot(y_seq)
plt.title('Time Series Batch: Training (past) vs Labels (future)')
plt.ylabel('Measurement')
plt.xlabel('Time')
plt.legend(['Training (x)', 'Labels (y)'], loc='upper left')

## Create the Recurrent Neural Network

*We will create the forecasting model using a Recurrent Neural Network, such as vanilla RNN, LSTM and GRU.*

*Network parameters: note that there is 3600 training examples and our sequence is 42 examples in length. As the batch-size is 256 sequences, each step in an epoch will run 256x42 examples. So the possibility of overtraining is huge unless we control these parameters carefully.*

In [ ]:
epochs = 50
steps_per_epoch = 60
warmup_steps = 7 # For custom loss function

### Loss Function

*We will use a custom `Mean Squared Error (MSE)` as the loss-function.*

The function will deal in a special way with events at the beginning of a sequence. As the model has seen input-events for a few time-steps only, so its generated output may be very inaccurate. Using the standard loss function which utilises the early time-steps may cause the model to distort its later output. We therefore give the model a "warmup-period" of `warmup_steps` time-steps where we don't use its accuracy in the loss-function.

In [ ]:
# The shape of both input tensors are: [batch_size, sequence_length, num_y_events]
def loss_mse_warmup(y_true, y_pred):
    # Ignore the "warmup" parts of the sequences
    y_true_slice = y_true[:, warmup_steps:, :]
    y_pred_slice = y_pred[:, warmup_steps:, :]
    mse = mean(square(y_true_slice - y_pred_slice))   
    return mse

### RNN Models

<font color="red">**Add more models here as needed**</font>

In [ ]:
def rnn_model_gru_sigmoid_1(num_x_events, num_y_events):
    model = Sequential()
    model.add(GRU(units=256,
                  return_sequences=True,
                  input_shape=(None, num_x_events,)))
    model.add(Dense(num_y_events, activation='sigmoid'))
    model.summary()
    return(model)

In [ ]:
def rnn_model_gru_sigmoid_2(num_x_events, num_y_events):
    model = Sequential()
    model.add(GRU(units=512,
                  return_sequences=True,
                  input_shape=(None, num_x_events,)))
    model.add(BatchNormalization())
    model.add(Dense(num_y_events, activation='sigmoid'))
    model.summary()
    return(model)

In [ ]:
def rnn_model_gru_layers(num_x_events, num_y_events):
    model = Sequential()
    model.add(GRU(units=512,
                  return_sequences=True,
                  input_shape=(None, num_x_events,)))
    model.add(GRU(units=256,
                  return_sequences=True))
    model.add(Dense(num_y_events, activation='sigmoid'))
    model.summary()
    return(model)

In [ ]:
# Alternatively, we can use a linear activation function to allow for the output 
# to take on arbitrary values. However, when more layers are in use, it may be 
# necessary to initialize weights with smaller values to avoid `NaN` values

from tensorflow.python.keras.initializers import RandomUniform

# This model will keep very small weights to start with
def rnn_model_gru_linear(num_x_events, num_y_events):
    init = RandomUniform(minval=-0.05, maxval=0.05)
    model = Sequential()
    model.add(GRU(units=512,
                    return_sequences=True,
                    input_shape=(None, num_x_events,)))
    model.add(Dense(num_y_events, activation='linear',
                    kernel_initializer=init))
    model.summary()
    return(model)

In [ ]:
# More custom models go here

### Create and compile the model

*Define a few optimizers to chose from. Note that this is the initial learning rate, which will be dynamic.*

In [ ]:
opt_sgd_1 = SGD(lr=0.01, momentum=0.0, nesterov=False)
opt_sgd_2 = SGD(lr=0.05, momentum=0.1, nesterov=False) # Good results with ReduceLROnPlateau
opt_rmsprop_1 = RMSprop(lr=0.005, rho=0.9, decay=0.5, epsilon=1e-07)
opt_rmsprop_2 = RMSprop(lr=0.01, rho=0.9, decay=0.1, epsilon=1e-07)
opt_adadelta_1 = Adadelta(lr=0.001, rho=0.95, epsilon=1e-07)
opt_adadelta_2 = Adadelta(lr=0.01, rho=0.99, epsilon=1e-07)
opt_adam_1 = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)
opt_adam_2 = Adam(lr=0.005, beta_1=0.85, beta_2=0.999, epsilon=1e-07)
opt_nadam = Nadam(lr=0.001, beta_1=0.7, beta_2=0.95, epsilon=1e-07)

# Start collecting history, in case we train the model iteratively
rnn_hist = start_hist()

model = rnn_model_gru_sigmoid_2(num_x_events, num_y_events)
model.compile(loss=loss_mse_warmup, optimizer=opt_sgd_2, metrics=[metrics.mae])
# model.compile(loss='mean_squared_error', optimizer=opt_sgd_2, metrics=[metrics.mae])

### Callback functions

*During training we want to save checkpoints and log the progress to TensorBoard so we create the appropriate callbacks for Keras*

In [ ]:
path_checkpoint = './gru_checkpoints/'

callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
        monitor='val_loss',
        verbose=1,
        save_weights_only=True,
        save_best_only=True)

callback_early_stopping = EarlyStopping(monitor='val_loss',
        patience=10, verbose=1)

callback_tensorboard = TensorBoard(log_dir='./gru_logs/',
        histogram_freq=0,
        write_graph=False)

# This callback reduces the learning-rate if the validation-loss has not improved as defined by `patience`.
# The learning-rate will be reduced by a `factor` (by multiplying) but no more than `min_lr`.
callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',
        factor=0.3,
        min_lr=1e-4,
        patience=5,
        min_delta = 1e-3,
        verbose=1)

keras_callbacks = [
        callback_early_stopping,
        #callback_tensorboard,
        callback_reduce_lr,
        callback_checkpoint
]

### Train the Recurrent Neural Network (RNN)

*Note that within each `epoch` batch-generator will randomly select sub-sequences from the training-set, controlled by `steps_per_epoch`.*

*Also note that the loss could become `NaN`, which can be resolved by restarting and running the Notebook again. But it may also be caused by your neural network architecture, learning-rate, batch-size, sequence-length, etc. in which case you may have to modify those settings.*

In [ ]:
%%time

# fix random seed for (near) reproducibility
np.random.seed(7)

# Fit the model
perform_indics = model.fit(x=generator,
        epochs=epochs,
        steps_per_epoch=steps_per_epoch,
        validation_data=validation_data,
        callbacks=keras_callbacks,
        verbose=1 # Use 0 or 2 to speed up
        )

# Add performance history
rnn_hist = collect_hist(rnn_hist, perform_indics.history)

*Plot training performance*

In [ ]:
plot_hist(rnn_hist, xsize=10, ysize=8)

### Load checkpoint

*As we used ModelCheckpoint callback, we can reload the last saved checkpoint, which had the best performance on the validation set*

In [ ]:
try:
    model.load_weights(path_checkpoint)
except Exception as error:
    print("Error trying to load checkpoint.")
    print(error)

## Performance

*Training performance*

In [ ]:
result = model.evaluate(x=np.expand_dims(x_train_scaled, axis=0),
                        y=np.expand_dims(y_train_scaled, axis=0))
# We have several metrics so we want to show their names
print()
for res, metric in zip(result, model.metrics_names):
    print("{0}: {1:.5f}".format(metric, res))

*Test performance*

In [ ]:
result = model.evaluate(x=np.expand_dims(x_test_scaled, axis=0),
                        y=np.expand_dims(y_test_scaled, axis=0))
# We have several metrics so we want to show their names
print()
for res, metric in zip(result, model.metrics_names):
    print("{0}: {1:.5f}".format(metric, res))

## Generate predictions

*This helper-function plots the predicted and true output-events*

In [ ]:
# Plot the predicted and true output-events.

def plot_comparison(start_idx, length=100, train=True, xlim=None, ylim=None):
    """
    :param start_idx: Start-index for the time-series.
    :param length: Sequence-length to process and plot.
    :param train: Boolean whether to use training- or test-set.
    """
    
    if train:
        # Use training-data.
        x = x_train_scaled
        y_true = y_train
    else:
        # Use test-data.
        x = x_test_scaled
        y_true = y_test
    
    # End-index for the sequences.
    end_idx = start_idx + length
    
    # Select the sequences from the given start-index and
    # of the given length.
    x = x[start_idx:end_idx]
    y_true = y_true[start_idx:end_idx]
    
    # Input-events for the model.
    x = np.expand_dims(x, axis=0)

    # Use the model to predict the output-events.
    y_pred = model.predict(x)
    
    # The output of the model is between 0 and 1.
    # Do an inverse map to get it back to the scale
    # of the original data-set.
    y_pred_rescaled = y_scaler.inverse_transform(y_pred[0])
    
    # For each output-event.
    for event in range(len(labels)):
        # Get the output-event predicted by the model.
        event_pred = y_pred_rescaled[:, event]
        
        # Get the true output-event from the data-set.
        event_true = y_true[:, event]

        # Make the plotting-canvas bigger.
        plt.figure(figsize=(15,5))
        
        # Plot and compare the two events.
        plt.plot(event_true, label='true')
        plt.plot(event_pred, label='pred')
        
        # Plot grey box for warmup-period.
        p = plt.axvspan(0, warmup_steps, facecolor='black', alpha=0.15)
        
        # Plot labels etc.
        if (xlim): plt.xlim(xlim)
        if (ylim): plt.ylim(ylim)
        plt.ylabel(labels[event])
        plt.legend()
        plt.show()

*Plot predicted output-events, which show only output-events and not the input-events used to predict the output-events. The `time-shift` between the input-events and the output-events is held fixed as defined in the `horizon` variable. So the x-axis merely shows how many time-steps of the input-events have been seen by the predictive model so far.*

*The prediction is not very accurate for the initial time period because the model has seen very little input-data.
The model needs to "warm up" first and so we ignore this "warmup-period" (shown as a grey box) in loss calculation.*

**Example from the training data**

*Change the ylim=(yfrom, yto) and xlim=(xfrom, xto) to zoom in on the relevant parts of the chart.*

In [ ]:
# If you do see anything, delete ylim parameter
plot_comparison(start_idx=0, length=3000, train=True, ylim=(20,250))

**Example from test data**

*Change the ylim=(yfrom, yto) and xlim=(xfrom, xto) to zoom in on the relevant parts of the chart.*

In [ ]:
# If you do see anything, delete ylim parameter
plot_comparison(start_idx=0, length=1500, train=False, ylim=(50, 320))

<br>**All done! Your report follows.**
<hr style="height:1px;border:none;color:#333;background-color:#333;" />

# RNN models, experiments and their results

## Reflection on experience gained in deep learning (as related to the entire module 3)

<font color="red">**Provide your reflection here.**</font>

*Summarise the lessons learnt in module 3 on Deep Learning. List or tabulate the models you have experimented with. Optionally, compare and contrast their characteistics and the methods of their training (e.g. in the same table). Discuss the strengths and weaknesses of each of the DL approaches used in this module. Any recommendations for the future?*
<br>

## RNN analysis and recommendation (as related to this notebook)

<font color="red">**Provide your analysis here.**</font>

*In a paragraph discuss your results with RNN models. Identify the strengths and weaknesses of the selected models. Which model would you recommend as a portfolio management solution and why.*
<br>

## Summary of RNN experimental results

<font color='red'>**Complete this table with the summary of all your experimental results.**</font><br>
**Work in the order of tasks listed at top of this notebook. Example of table entry has been included.**

*Performance is measured as MAE comparing the the original and poredicted stock price.*

| Model# | Run# | Batch | Seq | Epochs | Steps/Epoch | Stopped/Epochs | Optimiser | LR reduce factor | Train MAE | Valid MAE |
| :- | :-: | :-: | :-: | :-: | :-: | :-: | :- | -: | -: | -: |
| rnn_model_gru_sigmoid | 1 | 256 | 42 | 50 | 100 | 50/50 | SGD(lr=0.05, momentum=0.1, nesterov=False) | 0.3 | 0.02274 | 0.03080 |
| xxx | xxx | xxx | xxx | xxx | xxx | xxx | *Replace the contents with your experimental results* | xxx | xxx | xxx |

## Details of RNN model training runs and test results
<font color="red">**Provide details of the best models, within each category of your tasks, their training runs and test results.**</font><br>
**Example of a training and test run record has been included.**

***

***rnn_model_gru_sigmoid, run #1:***<pre>
*Brief model description goes here.*
*Also what is special about this run.*

**Data**
stocks = ['Date', 'Day', 'AMZN', 'GOOGL', 'HPQ']
labels = ['HPQ']
horizon = 2

**Performance**
Train MAE: 0.02274
Valid MAE: 0.03080

**Training history and results**</pre>

![stock_hpq_perf_loss.png](attachment:stock_hpq_perf_loss.png)

![stock_hpq_perf_mae.png](attachment:stock_hpq_perf_mae.png)

<center><i>Prediction on Training Data</i></center>

![stock_hpq_pred_train.png](attachment:stock_hpq_pred_train.png)

<center><i>Prediction on Test Data</i>

![stock_hpq_pred_test.png](attachment:stock_hpq_pred_test.png)

***

***Another model, run #2:***<pre>
</pre>

***